In [ ]:
!nvidia-smi

Tue Jul  7 10:16:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!mkdir coco
%cd coco
!wget http://images.cocodataset.org/zips/train2017.zip && unzip train2017.zip && rm train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip && unzip val2017.zip && rm val2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip && unzip annotations_trainval2017.zip && rm annotations_trainval2017.zip
%cd ../

/content/coco
--2020-07-07 10:17:11--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.99.75
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.99.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip        43%[=======>            ]   7.82G  16.3MB/s    eta 10m 56s

In [ ]:
!pip install webcolors yacs
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install tensorboardX
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
!git clone -b mask https://github.com/anslt/Yet-Another-EfficientDet-Pytorch.git

In [ ]:
%cd Yet-Another-EfficientDet-Pytorch/
%mkdir datasets
%mv /content/coco ./datasets

In [ ]:
!python setup.py build develop

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime, os
%tensorboard --logdir /content/Yet-Another-EfficientDet-Pytorch/logs/coco/tensorboard

In [ ]:
%cd /content/Yet-Another-EfficientDet-Pytorch/
!git pull origin mask

/content/Yet-Another-EfficientDet-Pytorch
From https://github.com/anslt/Yet-Another-EfficientDet-Pytorch
 * branch            mask       -> FETCH_HEAD
Already up to date.


In [ ]:
#!rm -rf "/content/drive/My Drive/res/coco"
# !python train_mask.py --config-file projects/efficientdet.yaml -c 0 -p coco --num_epochs 6 --num_workers 4 --head_only True --lr 1e-5 --batch_size 24 --load_weights "/content/drive/My Drive/efficientdet_weights/efficientdet-d0.pth"  --num_epochs 4 --saved_path "/content/drive/My Drive/efficientdet_weights" --saved_path "/content/drive/My Drive/res"
# mask_only
!python train_mask.py --config-file projects/efficientdet.yaml -c 0 -p coco --num_epochs 6 --num_workers 4 --mask_only True --lr 1e-5 --batch_size 5 --load_weights "/content/drive/My Drive/efficientdet_weights/efficientdet-d0.pth" --saved_path "/content/drive/My Drive/res"

In [ ]:
!python test_mask.py -n 0 --batch_size 5 -p coco -cfg projects/efficientdet.yaml -c 0 -w "/content/drive/My Drive/res/efficientdet-d0_0_0.pth" --data_path "/content/Yet-Another-EfficientDet-Pytorch/datasets/" --cuda True

running coco-style evaluation on project coco, weights /content/drive/My Drive/res/efficientdet-d0_0_0.pth...
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
0it [00:00, ?it/s]/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)
1000it [04:38,  3.59it/s]
Traceback (most recent call last):
  File "test_mask.py", line 132, in <module>
    main()
  File "test_mask.py", line 126, in main
    output_folder=output_folder,
  File "/content/Yet-Another-EfficientDet-Pytorch/maskrcnn_benchmark/engine/inference.py", line 447, in inference
    coco_results["segm"] = prepare_for_coco_segmentation(predictions, dataset)
  File "/content/Yet-Another-EfficientDet-Pytorch/maskrcnn_benchmark/engine/inference.py", line 123, in prepare_for_coco_segmentation
    masks = masker(masks, pred